In [1]:
from __future__ import annotations

from dotenv import load_dotenv
load_dotenv()

import os
import multiprocess
import pandas as pd
import tqdm.notebook as tqdm
import loaders

from typing import Any

In [2]:
tickers = loaders.ticker.load_all(market='stocks', type='CS')
len(tickers)

5642

In [3]:
def load_ticker_details(ticker: str) -> dict[str, Any] | None:
    import loaders

    try:
        details = loaders.ticker.load_details(ticker)
        return details
    except Exception as e:
        return None

In [4]:
CPU_COUNT = os.cpu_count()
ctx = multiprocess.get_context('spawn') # type: ignore

with ctx.Pool(CPU_COUNT) as pool:
    iterator = pool.imap(load_ticker_details, tickers)
    tickers_with_details = list(tqdm.tqdm(iterator, total=len(tickers)))

tickers_with_details = ([
    ticker_with_details 
    for ticker_with_details 
    in tickers_with_details 
    if ticker_with_details is not None
])

  0%|          | 0/5642 [00:00<?, ?it/s]

In [5]:
df = pd.DataFrame(tickers_with_details)
df.to_csv('../../stock-tickers.csv', index=False)

df

,ticker,name,listed_date,sic_code,market_cap,weighted_shares
0,A,Agilent Technologies Inc.,1999-11-18,3826,4.372096e+10,296072040.0
1,AA,Alcoa Corporation,2016-10-18,3334,8.202807e+09,176937165.0
2,AAC,Ares Acquisition Corporation,2021-03-25,6770,1.263750e+09,125000000.0
3,AACI,Armada Acquisition Corp. I Common Stock,2021-11-10,6770,2.104085e+08,20709500.0
4,AADI,"Aadi Bioscience, Inc. Common Stock",2021-08-27,2834,3.046950e+08,24395117.0
...,...,...,...,...,...,...
5637,ZVSA,"ZyVersa Therapeutics, Inc. Common Stock",2022-12-13,2834,1.462189e+07,9081922.0
5638,ZWS,Zurn Elkay Water Solutions Corporation,2012-03-29,3560,3.952374e+09,177794600.0
5639,ZYME,Zymeworks Inc.,2017-04-28,2834,4.819218e+08,62996314.0
5640,ZYNE,"Zynerba Pharmaceuticals, Inc",2015-08-05,2834,2.550799e+07,47062711.0
